In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [ ]:
# selenium을 활용한 Instagram 게시글 접근 코드

# 로그인 정보
_INSTAGRAM_ID = 'YOUR ID'
_INSTAGRAM_PW = 'YOUR PW'

# 옵션 설정
options = webdriver.ChromeOptions()
user_agent= "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.4.1.6 Safari/537.36"
options.add_argument('user-agent=' + user_agent)

dr = webdriver.Chrome(options=options)
dr.set_window_size(820, 1000)

# 인스타그램 로그인
dr.get('https://www.instagram.com/')
time.sleep(3)

input_id = dr.find_element(By.CSS_SELECTOR, 'input[name="username"]')
input_pw = dr.find_element(By.CSS_SELECTOR, 'input[name="password"]')
button_login = dr.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
input_id.send_keys(_INSTAGRAM_ID)
input_pw.send_keys(_INSTAGRAM_PW)
button_login.click()
time.sleep(5)

# 팝업 처리 (나중에 저장하기)
button = dr.find_elements(By.XPATH, "//*[contains(text(), '나중에 하기')]")
if len(button) > 0:
    button[0].click()

# 팝업 처리 (홈 화면에 추가)
time.sleep(5)
button = dr.find_elements(By.XPATH, "//*[contains(text(), '취소')]")
if len(button) > 0:
    button[0].click()

# 팝업 처리 (알림 설정)
#time.sleep(5)
#button = dr.find_elements(By.XPATH, "//*[contains(text(), '나중에 하기')]")
#if len(button) > 0:
#    button[0].click()

# 맛집 검색 페이지로 이동 (키워드 변경해가면서 사용 ex: 여수맛집, 제주도맛집)
dr.get('https://www.instagram.com/explore/search/keyword/?q=%23여수맛집')
time.sleep(5)


In [ ]:
from crawling_utils import select_first

# 데이터 수집
select_first(dr) # 첫 게시글 열기

In [5]:
results = [] # 크롤링 결과를 저장할 리스트

In [ ]:
from crawling_utils import move_next, get_content

# 크롤링 함수 - target 개수만큼 게시글을 넘어가며, 내용을 크롤링함
def crawling_instagram(target):
    cnt = 0
    while cnt < target:
        try:
            data = get_content(dr)
            if data is not None:
                results.append(data)
                cnt += 1
                print(cnt)
            move_next(dr)

        except:
            time.sleep(7)
            move_next(dr)

In [ ]:
crawling_instagram(2)

1
2


In [27]:
# 결과 리스트를 데이터프레임으로 변환
results_df = pd.DataFrame(results)
results_df.columns = ['content', 'likes','hashtags', 'comments', 'restaurant name', 'address', 'date']
results_df.drop_duplicates(subset=['content'], inplace=True) # 광고 등 중복 게시글 제거

In [28]:
from crawling_utils import clear_emoji

# 이모지 제거 및 빈칸 삭제
# 데이터가 깨지지 않고 저장될 수 있도록 함
# 이모지가 포함되어 있으면 cp949로 encoding 할 수 없음
results_df = clear_emoji(results_df)

In [29]:
# 결과 저장 (임의의 이름)
results_df.to_csv('region_restaurant.csv', encoding='cp949', index=False)